In [4]:
import sys
#import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
import skimage
from pathlib import Path
import shutil
from base64 import b64encode
from base64 import b64decode

plt.rcParams["figure.figsize"] = (10,10)

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [5]:
import numpy as np
import os
import configparser
import argparse
import cv2
import re
from typing import Set, List, Optional
import traceback
import sys
import time

import configparser
from pero_ocr.document_ocr.layout import PageLayout
from pero_ocr.document_ocr.page_parser import PageParser


def get_value_or_none(config, section, key):
    if config.has_option(section, key):
        value = config[section][key]
    else:
        value = None
    return value

def create_dir_if_not_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
config_path = "/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_lay_out_ocr/config.ini"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # suppress tensorflow warnings on loading models

config = configparser.ConfigParser()
config.read(config_path)
    
#define parser instance
page_parser = PageParser(config, config_path=os.path.dirname(config_path))
       
#Read in paths
input_image_path = get_value_or_none(config, 'PARSE_FOLDER', 'INPUT_IMAGE_PATH')
input_xml_path = get_value_or_none(config, 'PARSE_FOLDER', 'INPUT_XML_PATH')

output_render_path = get_value_or_none(config, 'PARSE_FOLDER', 'OUTPUT_RENDER_PATH')
output_xml_path = get_value_or_none(config, 'PARSE_FOLDER', 'OUTPUT_XML_PATH')


run_layout_parser = config['PAGE_PARSER'].getboolean('RUN_LAYOUT_PARSER')
run_line_cropper = config['PAGE_PARSER'].getboolean('RUN_LINE_CROPPER')
run_ocr = config['PAGE_PARSER'].getboolean('RUN_OCR')


if run_ocr and not run_line_cropper:
    raise Exception( f"If [PAGE_PARSER] RUN_OCR is 'yes', then [PAGE_PARSER] RUN_LINE_CROPPER should also be set to 'yes' in {config_path}" )

if (not run_layout_parser) and ( run_ocr ) and (input_xml_path is None):
    raise Exception( f"[PARSE_FOLDER] INPUT_XML_PATH should be specified if [PAGE_PARSER] RUN_LAYOUT_PARSER is set to 'no' and [PAGE_PARSER] RUN_OCR is set to 'yes'"  )
    
if output_render_path is not None:
    create_dir_if_not_exists(output_render_path)

if output_xml_path is not None:
    create_dir_if_not_exists(output_xml_path)

if input_image_path is None:
    raise Exception( f"[PARSE_FOLDER] INPUT_IMAGE_PATH has to be specified. Missing in {config_path}")  
else:
    print(f'Reading images from {input_image_path}.')
    images_to_process = [f for f in os.listdir(input_image_path) if
                         os.path.splitext(f)[1].lower() in ['.jpg', '.jpeg', '.png', '.tif']]
    ids_to_process = [os.path.splitext(os.path.basename(file))[0] for file in images_to_process]
    
if input_xml_path is not None:
    print(f'Reading page xml from {input_xml_path}')
    xml_to_process = [f for f in os.listdir(input_xml_path) if
                      os.path.splitext(f)[1] == '.xml']

for index, (file_id, image_file_name) in enumerate(zip(ids_to_process, images_to_process)):
    print("Processing {file_id}".format(file_id=file_id))
    
    image = cv2.imread(os.path.join(input_image_path, image_file_name), 1)
    if image is None:
        raise Exception(f'Unable to read image "{os.path.join(input_image_path, image_file_name)}"')
    
    #when you only want ocr and no lay_out, you read the page-xml:
       
    if input_xml_path:
        page_layout = PageLayout(file=os.path.join(input_xml_path, file_id + '.xml'))
    else:
        page_layout = PageLayout(id=file_id, page_size=(image.shape[0], image.shape[1]))

    page_layout = page_parser.process_page(image, page_layout)

    #reading in page_xml format:
    if output_xml_path is not None:
        page_layout.to_pagexml(os.path.join(output_xml_path, file_id + '.xml'))

    if output_render_path is not None:
        page_layout.render_to_image(image)
        cv2.imwrite(os.path.join(output_render_path, file_id + '.jpg'), image, [int(cv2.IMWRITE_JPEG_QUALITY), 70])         




INFO:tensorflow:Restoring parameters from /notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_lay_out_ocr/layout/ParseNet_exported


INPUT (8, 32, ?, 3)


height 16
CONV BLOCK 0 (8, 16, ?, 24)


height 8
CONV BLOCK 1 (8, 8, ?, 48)
height 8


1D (8, ?, 256)
LOGIT SHAPE (8, ?, 177)


INFO:tensorflow:Restoring parameters from /notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_lay_out_ocr/ocr/checkpoint_646000.ckpt
Reading images from /notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_lay_out_ocr/example/images.
Processing 1
med height: 16.540254592895508, ds: 4, recomp   ute?, False


In [5]:
import os
file='/DIR/image.1.png'
os.path.splitext(os.path.basename(file))[0]


'image.1'

In [4]:
#output_xml_path = get_value_or_none(config, 'PAGE_PARSER', 'RUN_LAYOUT_PARSER')

Processing 1


/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/code/pero-ocr/pero_ocr/document_ocr/crop_engine.py:99: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reverse_line_mapping" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at /notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/code/pero-ocr/pero_ocr/document_ocr/crop_engine.py (101)

File "../pero_ocr/document_ocr/crop_engine.py", line 101:
    def reverse_line_mapping(self, forward_mapping, sample_positions, sampled_values):
        backward_mapping = np.zeros_like(sample_positions)
        ^

  @jit
/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/code/pero-ocr/pero_ocr/document_ocr/crop_engine.py:99: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "reverse_line_mapping" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "../

In [6]:
input_image_path

'/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_lay_out/example/images'

In [7]:
images_to_process

['1.png']

In [9]:
print("something")

something


# END

In [18]:
config.read( config_path )

['/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_test/config.ini']

In [21]:
config[ 'PARSE_FOLDER' ]['INPUT_IMAGE_PATH']

'/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_test/example/images'

In [28]:
output_line_path

'/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_test/example/images'

In [15]:
input_image_path

'/notebook/nas-trainings/arne/OCCAM/pero_ocr_no_django/MODELS/model_occam_test/example/images'

In [16]:
input_xml_path

In [17]:
input_logit_path

In [ ]:
if input_image_path is not None:
    image = cv2.imread(os.path.join(input_image_path, image_file_name), 1)
    if image is None:
        raise Exception(f'Unable to read image "{os.path.join(input_image_path, image_file_name)}"')
else:
    image = None

if input_xml_path:
    page_layout = PageLayout(file=os.path.join(input_xml_path, file_id + '.xml'))
else:
    page_layout = PageLayout(id=file_id, page_size=(image.shape[0], image.shape[1]))